In [10]:
# import subprocess

# # Replace "LKH-2.exe" with the actual path or name of your .exe file
# # Replace "E3k.0.par" with the actual parameter for your .exe file
# result = subprocess.check_output(["LKH-2.exe", "E3k.0.par"], stdin=subprocess.DEVNULL)

# # Decode the output assuming it's in UTF-8 encoding
# output = result.decode("utf-8")

# # Print the captured output
# print(output)


In [18]:
import re
import subprocess
import numpy as np
class lkh:
    def __init__(self , dist_mat_file_name):
        self.dist_mat_file_name = dist_mat_file_name
        

    def solve(self):
        '''
        solves the TSP instance using .exe file
        '''
        # Generate the .par file
        self.generate_parameter_file(self.dist_mat_file_name)

        # Replace "E3k.0.par" with the actual parameter for your .exe file
        result = subprocess.check_output(["LKH-2.exe", "E3k.0.par"], stdin=subprocess.DEVNULL)

        # Decode the output assuming it's in UTF-8 encoding
        output = result.decode("utf-8")

        # Print the captured output
        print("Objective function value : ",self.get_obj_val(output))

        #Print the routing
        print("The routing : " , self.get_routing("br17.txt"))

    
    def get_obj_val(self , output):
        '''
        returns the objective value from the  long output value
        '''
        # Assuming `output` contains the captured output
        cost_min_match = re.search(r'Cost\.min = (\d+)', output)

        if cost_min_match:
            cost_min_value = int(cost_min_match.group(1))
            return cost_min_value
        else:
            print("Cost.min not found in the output.")
            return 0    
    
    def get_routing(self , filename):
        '''
        returns the routing in a numpy array
        '''
        start_keyword = 'TOUR_SECTION'
        end_keyword = '-1'
        with open(filename, 'r') as file:
            lines = []
            read_flag = False
            for line in file:
                if start_keyword in line:
                    read_flag = True
                    continue  # skip this line
                if read_flag:
                        if end_keyword in line:
                            read_flag = False
                            break
                        else:
                            lines.append(line.strip())

        lines = np.array(lines)
        return lines
    
    def generate_parameter_file(self , filename):
        runs = 1
        filename_atsp = filename + ".atsp"
        filename_txt = filename + ".txt"  
        filename_par = filename + ".par"

        with open(filename_par, 'w') as f:
            f.write(f"PROBLEM_FILE = {filename_atsp}\n")
            f.write(f"OUTPUT_TOUR_FILE = {filename_txt}\n")
            f.write(f"OPTIMUM = 40634081 \n")
            f.write(f"INITIAL_PERIOD = 1000 \n")
            f.write(f"MAX_CANDIDATES = 6 \n")
            f.write(f"MAX_TRIALS = 1000 \n")
            f.write(f"MOVE_TYPE = 6 \n")
            f.write(f"PATCHING_C = 6 \n")
            f.write(f"PATCHING_A = 5 \n")
            f.write(f"RECOMBINATION = GPX2 \n")
            f.write(f"RUNS = {runs} \n")
    
    def read_pickle(self,pickle_path):
        # Open the pickle file in read-binary mode
        with open(pickle_path, 'rb') as f:
            # Load the object from the pickle file
            pic_obj = pickle.load(f)
        
        pic_obj = self.round_up(self.dist_mat)


        # Do something with the object
        print(pic_obj)
        
        return pic_obj
    
    def generate_atsp_file(self , dist_mat, filename):   
        '''
        input : 
        a symmentric matrix  
        directory of a .tsp file you want to generate
        number of cities in the TSP

        Output : 
        a .tsp file generated in the filename directory
        '''
        # Defining the number of cities
        num_cities = len(dist_mat)

        # Write the coordinates to a .tsp file
        fname = filename.split("/")[-1].split(".")[0]
        with open(filename, 'w') as f:
            f.write(f"NAME: {fname}\n")
            # f.write("COMMENT: Randomly generated TSP instance\n")
            f.write("TYPE: TSP (M.~Hofmeister)\n")
            f.write(f"DIMENSION: {num_cities}\n") #becase the distance matrix doubles in size when converted to a symmentric instance
            f.write("EDGE_WEIGHT_TYPE: EXPLICIT\n")
            f.write("EDGE_WEIGHT_FORMAT: FULL_MATRIX\n")
            # f.write("NODE_COORD_SECTION\n")
            f.write("DISPLAY_DATA_TYPE: NO_DISPLAY\n")
            f.write("EDGE_WEIGHT_SECTION\n")
            for i in range(len(dist_mat)):
                for j in range(len(dist_mat)):
                f.write(f'{str(dist_mat[i][j])}\t')
                f.write("\n")
            f.write("EOF\n")

    

    

s1 = lkh("warmze_31606-12_dienstag_briefe1_distmatrix (1)")
s1.solve()
        

Objective function value :  39
The routing :  ['1' '12' '14' '3' '11' '13' '10' '2' '17' '8' '9' '5' '4' '16' '6' '15'
 '7']
